Installation of necessary packages (and creation of a conda env):
```
./install_main.sh 
```

Process results obtained via running command:
```
bash -i run_qa_main.sh
```

In [1]:
SEED_QTY=3
OUTPUT_PREF='results_qa/output_res_'
#OUTPUT_PREF='../test_accelerate/results/output_res_'

In [2]:
#sync_step_arr = [1, 2, 4, 8, 16]
sync_step_arr = [1, 2, 4, 8, 16, 32, 64, 128, 256]

res_keys = ['1gpu'] + \
        [f'accum_steps_{k}' for k in sync_step_arr] + \
        [f'nosync_steps_{k}' for k in sync_step_arr]

In [3]:
import json

res_dict = {k : [] for k in res_keys}

for train_qty in [4000, 40000]:
    for rk in res_keys:
        one_res = {}
        
        for seed in range(SEED_QTY):
            tmp = json.load(open(f'{OUTPUT_PREF}{train_qty}_{rk}/{seed}/all_results.json'))
            if seed == 0:
                for mk in tmp.keys():
                    one_res[mk] = []

            for mk in tmp.keys():
                one_res[mk].append(tmp[mk])
                 
        res_dict[rk].append(one_res)
    

In [4]:
res_dict['1gpu']

[{'eval_exact_match': [69.65941343424787,
   69.78240302743615,
   69.57426679280984],
  'eval_f1': [80.26637915296253, 80.08236129993176, 79.77908879505684],
  'eval_train_time': [294.5597097873688,
   294.7031977176666,
   294.03918075561523]},
 {'eval_exact_match': [80.88930936613056, 81.2961210974456, 81.20151371807],
  'eval_f1': [88.4562154274226, 89.04475780185317, 89.05548149018428],
  'eval_train_time': [2748.6899106502533,
   2749.4790568351746,
   2739.752244949341]}]

In [5]:
tmp

{'eval_exact_match': 0.07568590350047304,
 'eval_f1': 6.14816425600873,
 'eval_train_time': 711.9177227020264}

In [6]:
import numpy as np

res_dict_str = {k : [] for k in res_keys}

for cidx, train_qty in enumerate([4000, 40000]):
    for rk in res_keys:
        speed_up = round(np.mean(res_dict['1gpu'][cidx]['eval_train_time']) / np.mean(res_dict[rk][cidx]['eval_train_time']), 1)
        f1 = str(round(np.mean(res_dict[rk][cidx]['eval_f1']), 1)) +'/' +str(round(np.max(res_dict[rk][cidx]['eval_f1']), 1))
        #print(rk, speed_up, f1)
        if rk != '1gpu':
            res_dict_str[rk].append(f'f1={f1} {speed_up}x')
        else:
            res_dict_str[rk].append(f'f1={f1}')  
            
            
res_dict_str

{'1gpu': ['f1=80.0/80.3', 'f1=88.9/89.1'],
 'accum_steps_1': ['f1=55.5/80.5 1.4x', 'f1=32.7/87.8 1.3x'],
 'accum_steps_2': ['f1=79.7/80.3 2.1x', 'f1=88.3/88.7 2.1x'],
 'accum_steps_4': ['f1=78.3/78.4 3.0x', 'f1=88.6/88.8 2.9x'],
 'accum_steps_8': ['f1=73.6/76.4 3.7x', 'f1=88.4/88.5 3.7x'],
 'accum_steps_16': ['f1=61.5/68.3 4.2x', 'f1=87.6/88.2 4.3x'],
 'accum_steps_32': ['f1=24.0/32.8 4.6x', 'f1=85.3/86.0 4.7x'],
 'accum_steps_64': ['f1=12.0/13.3 4.8x', 'f1=82.4/83.8 4.9x'],
 'accum_steps_128': ['f1=8.5/9.0 4.9x', 'f1=72.1/75.2 5.0x'],
 'accum_steps_256': ['f1=8.5/9.0 4.9x', 'f1=40.8/56.6 5.1x'],
 'nosync_steps_1': ['f1=80.1/80.3 1.2x', 'f1=88.8/88.8 1.1x'],
 'nosync_steps_2': ['f1=80.0/80.3 1.8x', 'f1=88.8/88.8 1.7x'],
 'nosync_steps_4': ['f1=80.1/80.2 2.4x', 'f1=88.8/88.8 2.4x'],
 'nosync_steps_8': ['f1=80.1/80.5 3.0x', 'f1=88.6/88.7 2.9x'],
 'nosync_steps_16': ['f1=80.0/80.2 3.4x', 'f1=88.7/88.8 3.3x'],
 'nosync_steps_32': ['f1=79.8/80.2 3.6x', 'f1=88.7/88.7 3.6x'],
 'nosync_steps_6

In [7]:
import pandas as pd
df=pd.DataFrame.from_dict(res_dict_str)

In [8]:
df

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,accum_steps_32,accum_steps_64,accum_steps_128,accum_steps_256,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16,nosync_steps_32,nosync_steps_64,nosync_steps_128,nosync_steps_256
0,f1=80.0/80.3,f1=55.5/80.5 1.4x,f1=79.7/80.3 2.1x,f1=78.3/78.4 3.0x,f1=73.6/76.4 3.7x,f1=61.5/68.3 4.2x,f1=24.0/32.8 4.6x,f1=12.0/13.3 4.8x,f1=8.5/9.0 4.9x,f1=8.5/9.0 4.9x,f1=80.1/80.3 1.2x,f1=80.0/80.3 1.8x,f1=80.1/80.2 2.4x,f1=80.1/80.5 3.0x,f1=80.0/80.2 3.4x,f1=79.8/80.2 3.6x,f1=78.1/79.1 3.8x,f1=34.0/56.3 3.9x,f1=34.0/56.3 4.0x
1,f1=88.9/89.1,f1=32.7/87.8 1.3x,f1=88.3/88.7 2.1x,f1=88.6/88.8 2.9x,f1=88.4/88.5 3.7x,f1=87.6/88.2 4.3x,f1=85.3/86.0 4.7x,f1=82.4/83.8 4.9x,f1=72.1/75.2 5.0x,f1=40.8/56.6 5.1x,f1=88.8/88.8 1.1x,f1=88.8/88.8 1.7x,f1=88.8/88.8 2.4x,f1=88.6/88.7 2.9x,f1=88.7/88.8 3.3x,f1=88.7/88.7 3.6x,f1=88.2/88.5 3.7x,f1=61.0/88.7 3.8x,f1=23.6/57.6 3.9x


In [9]:
#df.to_csv('results.tsv')

In [11]:
#!cat results.tsv